# League of Legends - first 10 minutes

## Analyze historical data and suggest best scenario to win League of Legends game.

![](https://osclasspoint.com/kaggle/lol-leader.jpg)

We will work with 10k data of previous games (first 10 minutes) and will try to analyze what are most important factors to win the game.

# Load libraries and data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = 500
plt.rcParams.update({'figure.figsize': (12, 5), 'figure.dpi': 100})
sns.set(style="whitegrid")

In [ ]:
df = pd.read_csv('/kaggle/input/league-of-legends-diamond-ranked-games-10-min/high_diamond_ranked_10min.csv')

## Data types and missing values

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.isnull().sum() > 0

Let's drop gameId as it does not bring any extra value to our data.

In [ ]:
df.drop('gameId', axis=1, inplace=True)

# Exploratory Data Analysis (EDA)
First let's check how wins are distributed

In [ ]:
plt.figure(figsize=(5,3))
sns.countplot(x=df.blueWins, palette=sns.color_palette(['r','b']))
plt.xticks([0,1], ['Red', 'Blue'])
plt.xlabel('')
plt.ylabel('')

Basically we have 5000 of red wins and 5000 of blue wins. As it should not matter on color at all, we will be looking on "blue" data only, as "red" ones should be just more likely reversed to blue data.

In [ ]:
# dropping all the columns starting with "red"
df = df.loc[:,~df.columns.str.startswith('red')]

## Correlation
Check how features are correlated to each other and what are most influencers to blueWins column.

In [ ]:
plt.figure(figsize=(15,15))
corrmat = df.corr()
corrmat = np.tril(corrmat)
corrmat[corrmat==0] = None
corrmat = corrmat.round(1)
labels = df.select_dtypes(include='number').columns.values
f, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(corrmat, annot=True, vmax=0.8,vmin=-0.8, cmap='seismic_r', xticklabels=labels,yticklabels=labels, cbar=False)
plt.legend('')

plt.show()

You may find pretty interesting correlations here, i.e. placing wards has no impact on game or strong correlation between blueTotalGold and blueKills.

Now check distributions and relations between features using scatterplot, where results are colorized by blueWins field.

In [ ]:
g = sns.PairGrid(data = df, vars=['blueWardsPlaced', 'blueKills', 'blueDeaths', 'blueAssists', 'blueAvgLevel', 'blueTotalExperience'], hue='blueWins', palette=sns.color_palette(['r', 'b']), hue_kws={"marker": ["D", "o"], "alpha": [0.3, 0.3]})
g.map_diag(plt.hist)
g.map_offdiag(plt.scatter)
g.add_legend()
plt.show()

Again there are interesting relations, like blueAvgLvl seems to be lower for blue team when red team wins.

Now let's sort correlation of features with blueWins column, blueDeaths field will not be used there.

In [ ]:
plt.figure(figsize=(15,6))
dfw = df.corr()['blueWins'].drop(['blueWins', 'blueDeaths'])
dfw = dfw.sort_values(ascending=False)

pal = sns.color_palette("Greens_d", len(dfw))
rank = dfw.argsort().argsort() 

sns.barplot(y=dfw.index, x=dfw, palette=np.array(pal[::-1])[rank])

plt.show()


Create ranking based on correlation for each feature on scale 0-10, where most important feature gets 10 points.

In [ ]:
dfw.apply(lambda x: round(round(20/dfw.max()*x)/2, 1))

# Conclusion
## Ok, so how to win?

We have shown different relations between data and how are they affected by win/loose. From perspective of blue team, **how to play to win the game**?

* farming golds is most important, even more important then kills
* leveling is more important then kills
* placing and destroying wards has very low impact on game results
* first blood, when considering it's just first kill, is pretty important for game win/lose
* assists are almost same important as kills, therefore it's not really needed to be pending for last hit on kill
* jungling, killing towers and heralds is not as important to win the game

## Summary
Basic rule is to farm golds on lane and not in jungle, level as fast as possible, kill enemies and help your mates to kill enemies. Do not rush towers, heralds, do not spent too much golds on wards & hunting enemy wards, don't jungle if it's not really needed.

## If you like this notebook, please upvote ;-)

## Thanks!